## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1075c4c10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1076fce50>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.messages import HumanMessage
resp = model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Resident Solution Architect")])
print(resp.content)

Hi Krish, it's nice to meet you! 

As a Resident Solution Architect, it sounds like you have a fascinating job.  

What kind of solutions do you typically work on?  

I'm curious to learn more about your role and the challenges you face.



In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Krish, and you are a Chief AI Engineer. \n\nIs there anything else you'd like me to remember about you?  I'm here to help! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 97, 'total_tokens': 138, 'completion_time': 0.074545455, 'prompt_time': 0.004229894, 'queue_time': 0.268702096, 'total_time': 0.078775349}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a8dccfb3-6f20-4285-8fa1-b6f32e969189-0', usage_metadata={'input_tokens': 97, 'output_tokens': 41, 'total_tokens': 138})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [9]:
!pip install -qq langchain_community

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(
    model,
    get_session_history
                                                )

In [12]:
config={"configurable":{"session_id":"chat1"}}

In [19]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Sourav and I am a Resident Solution Architect")],
    config=config
)

In [25]:
response.content

"Hello Sourav, it's a pleasure to meet you! \n\nA Resident Solution Architect is a very interesting role.  What kind of solutions are you currently working on? \n\nI'm always fascinated to hear about the challenges and successes people face in designing and implementing solutions.\n"

In [26]:
resp = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)
print(resp.content)

Your name is Sourav.  

How can I help you today, Sourav?  



In [27]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to know!\n"

In [28]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
print(response.content)

Hi John, it's nice to meet you! Is there anything I can help you with today?



In [29]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
print(response.content)

You said your name is John earlier!  😊  Did you forget? 




### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [30]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [31]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Sourav")]})

AIMessage(content="Hello Krish, it's nice to meet you!  \n\nI'm ready to answer your questions to the best of my ability. What can I help you with? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 31, 'total_tokens': 71, 'completion_time': 0.072727273, 'prompt_time': 0.002328621, 'queue_time': 0.264734911, 'total_time': 0.075055894}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--3b72d09b-25bf-448c-98b8-5c368c1bf5c7-0', usage_metadata={'input_tokens': 31, 'output_tokens': 40, 'total_tokens': 71})

In [32]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [37]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Sourav")],
    config=config
)

print(response.content)

Hi Sourav, it's great to meet you!  

What can I help you with today?  😄 



In [38]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

print(response.content)

Your name is Sourav. 😊  

Is there anything else I can help you with?



In [39]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [41]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Bengali"})
print(response.content)

নমস্কার Krish,

আমি আপনার সাহায্যকারী হিসেবে এখানে আছি। আপনার কোন প্রশ্ন আছে? 😊




Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [46]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [49]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Bengali"},
    config=config
)
repsonse.content

'নমস্কার কৃষ্ণ! 😊 \n\nআপনি কেমন আছেন? আমি আপনার জন্য কিছু করতে পারি? \n'

In [50]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Bengali"},
    config=config,
)

In [51]:
response.content

'আপনার নাম কৃষ্ণ। 😊  \n\n\n\n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [54]:
from langchain_core.messages import SystemMessage,trim_messages,AIMessage,HumanMessage
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [55]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your favorite ice cream flavor. \n\nWhat's your favorite ice cream flavor? 😊\n"

In [56]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked me "what is 2 + 2". 😊  \n\n\n\nLet me know if you want to try another one!\n'

In [57]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [58]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have memory of past conversations or any personal information about you, including your name.\n\nIf you'd like to tell me your name, I'd be happy to know! 😊\n"

In [59]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊 \n\nWhat's on your mind?  \n\n"